SegFormer-ft on ATR Dataset
====


**SegFormer: Simple and Efficient Design for Semantic Segmentation with Transformers**
 * Original SegFormer paper: https://arxiv.org/abs/2105.15203

![SegFormer](../assets/segformer_overview.png)

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from transformers import SegformerImageProcessor
from transformers import AutoModelForSemanticSegmentation


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = SegformerImageProcessor.from_pretrained(
    "mattmdjaga/segformer_b2_clothes"
)
model = AutoModelForSemanticSegmentation.from_pretrained(
    "mattmdjaga/segformer_b2_clothes"
)

model.eval().to(device);

/home/pyml/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
